In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 24 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

04 April 2023 ; 01:25:34


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,1,1
1,Bobby Portis,PF,27,MIL,67,20,25.9,5.6,11.3,0.499,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1
4,Goran Dragić,PG,36,MIL,55,0,14.7,2.4,5.6,0.421,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1
6,Jae Crowder,SF,32,MIL,14,0,17.9,1.6,3.9,0.426,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1
7,Jevon Carter,PG,27,MIL,78,37,22.3,2.9,6.9,0.424,...,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.9,65,0,BOS,18.2,2.523077,7.064615,14.7,2,2.1,4.415385,0.308970
1,Immanuel Quickley,28.7,78,18,NYK,15.9,2.102564,6.623077,14.3,5,1.0,4.205128,0.149442
2,Bobby Portis,25.9,67,20,MIL,17.6,1.346269,6.119403,13.8,1,1.0,2.814925,0.134784
3,Malik Monk,22.4,74,0,SAC,16.7,1.108108,3.545946,13.6,3,1.8,2.327027,0.089607
4,Austin Reaves,28.4,60,18,LAL,14.5,1.503333,6.560000,12.5,7,0.3,4.783333,0.087272


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)